In [27]:
# ExerciseTwo
# Student Name: Peisong Yang
# Student ID: 19031379

#import packages
import numpy as np
import pandas as pd
from pymongo import MongoClient
import json
from sklearn.cluster import KMeans

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# setting the db path
path = "D:/UCL MSc/Big Data"
rawdb = "/iftbigdatacoursework2019/000.DataBases/DBs/SP500Static/SP500Data.json"

In [3]:
# get data from raw db
with open(path+rawdb, 'r') as f:
    jsonData = json.loads(f.read())

# create mongodb client and database
myclient = MongoClient('mongodb://localhost:27017/')
mydb = myclient['SP500Data']
mycol = mydb['Static']

In [22]:
# insert data into my collections
mycol.insert_many(jsonData)

# and print one record
print(mycol.find_one())

{'_id': ObjectId('5e1c795241a2f618670392cc'), 'Symbol': 'MMM', 'Security': '3M Company', 'SECfilings': 'reports', 'GICSSector': 'Industrials', 'GICSSubIndustry': 'Industrial Conglomerates', 'Price': 168.76, 'DividendYield': 3.3, 'MarketCap': 97084, 'PERatio': 20, 'PayoutRatio': 66.3, 'Beta': 1.04}


In [24]:
# check that SP500Data is in the databases
myclient.list_database_names()

['SP500', 'SP500Data', 'admin', 'config', 'local']

In [38]:
# query and play a little bit

print("find stock name in financials sectors:")
print([i['Security'] for i in mycol.find({'GICSSector': "Financials"})])

print("\nlimit the number or queries in 10:")
print([i['Security'] for i in mycol.find({'GICSSector': "Financials"}).limit(10)])

print("\nfind stock number with beta equal to 1:")
print([i['Security'] for i in mycol.find({'Beta': {"$eq": 1}})])

find stock name in financials sectors:
['Affiliated Managers Group Inc', 'AFLAC Inc', 'Allstate Corp', 'American Express Co', 'American International Group', 'Ameriprise Financial', 'Aon plc', 'Arthur J. Gallagher & Co.', 'Assurant', 'Bank of America Corp', 'The Bank of New York Mellon Corp.', 'BB&T Corporation', 'Berkshire Hathaway', 'BlackRock', 'Capital One Financial', 'Cboe Global Markets', 'Charles Schwab Corporation', 'Chubb Limited', 'Cincinnati Financial', 'Citigroup Inc.', 'Citizens Financial Group', 'CME Group Inc.', 'Comerica Inc.', 'Discover Financial Services', 'E*Trade', 'Everest Re Group Ltd.', 'Fifth Third Bancorp', 'First Republic Bank', 'Franklin Resources', 'Goldman Sachs Group', 'Hartford Financial Svc.Gp.', 'Huntington Bancshares', 'Intercontinental Exchange', 'Invesco Ltd.', 'Jefferies Financial Group', 'JPMorgan Chase & Co.', 'KeyCorp', 'Lincoln National', 'Loews Corp.', 'M&T Bank Corp.', 'Marsh & McLennan', 'MetLife Inc.', "Moody's Corp", 'Morgan Stanley', 'MSCI

In [4]:
# now take all data from database and convert data into dataframe

# fullData = pd.DataFrame(jsonData)
fullData = mycol.find({})
fullData = pd.DataFrame(fullData)

# and close connaction to mongodB
myclient.close()

In [5]:
# first look at the data
print(fullData.info())
fullData.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
_id                505 non-null object
Symbol             505 non-null object
Security           505 non-null object
SECfilings         505 non-null object
GICSSector         505 non-null object
GICSSubIndustry    505 non-null object
Price              501 non-null object
DividendYield      501 non-null object
MarketCap          501 non-null object
PERatio            501 non-null object
PayoutRatio        501 non-null object
Beta               501 non-null object
dtypes: object(12)
memory usage: 47.5+ KB
None


,_id,Symbol,Security,SECfilings,GICSSector,GICSSubIndustry,Price,DividendYield,MarketCap,PERatio,PayoutRatio,Beta
0,5e1c795241a2f618670392cc,MMM,3M Company,reports,Industrials,Industrial Conglomerates,168.76,3.3,97084,20,66.3,1.04
1,5e1c795241a2f618670392cd,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,81.27,1.6,143636,0,0,1.06
2,5e1c795241a2f618670392ce,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,77.75,5.4,114951,28.5,153,0.84
3,5e1c795241a2f618670392cf,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,178.73,0,8110,31.3,0,1.5
4,5e1c795241a2f618670392d0,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,184.07,1.6,117278,24.6,39,0.99


In [6]:
# what are the SECfilings?
print(f"SECfilings: {set(fullData['SECfilings'])}")

# what about the sectors?
print(f"\nSectors: {set(fullData['GICSSector'])};\nThe num of sectors is: {len(set(fullData['GICSSector']))}")


SECfilings: {'reports'}

Sectors: {'Information Technology', 'Health Care', 'Consumer Discretionary', 'Energy', 'Utilities', 'Real Estate', 'Consumer Staples', 'Industrials', 'Financials', 'Communication Services', 'Materials'};
The num of sectors is: 11


In [7]:
# it seems the types are all object, convert the numerical ones to float64

# before transformation, substitute 'NA' with np.nan
numericalVar = ['Price','DividendYield','MarketCap','PERatio','PayoutRatio','Beta']
fullData[fullData[numericalVar] == 'NA'] = np.nan

# type converting
fullData[numericalVar] = fullData[numericalVar].astype('float64')

# check the data type again
fullData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
_id                505 non-null object
Symbol             505 non-null object
Security           505 non-null object
SECfilings         505 non-null object
GICSSector         505 non-null object
GICSSubIndustry    505 non-null object
Price              496 non-null float64
DividendYield      496 non-null float64
MarketCap          496 non-null float64
PERatio            496 non-null float64
PayoutRatio        496 non-null float64
Beta               496 non-null float64
dtypes: float64(6), object(6)
memory usage: 47.5+ KB


In [8]:
# now we are happy with the data type, now let's clean the data

# drop the _id first
fullData = fullData.drop(labels='_id', axis=1)

# drop nan
fullData = fullData.dropna(subset=numericalVar)

# describe numerical variables
fullData[numericalVar].describe()

,Price,DividendYield,MarketCap,PERatio,PayoutRatio,Beta
count,496.000000,496.000000,4.960000e+02,496.000000,496.000000,496.000000
mean,123.752500,2.011492,5.479029e+04,35.916734,95.770968,0.947177
std,170.422879,1.710765,1.119090e+05,292.509322,1331.748517,0.376390
min,9.140000,0.000000,2.939000e+03,-531.500000,-1280.000000,-0.110000
25%,47.977500,0.700000,1.282475e+04,12.100000,1.150000,0.720000
50%,86.960000,1.800000,2.319200e+04,20.150000,29.900000,0.970000
75%,141.157500,2.900000,4.824525e+04,31.450000,54.250000,1.200000
max,2032.230000,12.100000,1.098974e+06,6415.200000,29565.200000,2.340000


In [9]:
# drop queries with negtive PERaito and PayoutRatio
fullData = fullData[(fullData['PERatio'] > 0) & (fullData['PayoutRatio'] > 0)]

fullData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373 entries, 0 to 504
Data columns (total 11 columns):
Symbol             373 non-null object
Security           373 non-null object
SECfilings         373 non-null object
GICSSector         373 non-null object
GICSSubIndustry    373 non-null object
Price              373 non-null float64
DividendYield      373 non-null float64
MarketCap          373 non-null float64
PERatio            373 non-null float64
PayoutRatio        373 non-null float64
Beta               373 non-null float64
dtypes: float64(6), object(5)
memory usage: 35.0+ KB


In [10]:
# now let's plot some figure and observe
# use plotly to do this
# pip install plotly
# if the figure doesn't come out, please trust this note and use jupyter notebook instead of jupyterlab
# $ jupyter labextension install @jupyterlab/plotly-extension
# $ jupyter trust ExerciseTwo.ipynb

import plotly
import plotly.graph_objs as go
import plotly.express as px

plotly.offline.init_notebook_mode(connected=True)

In [11]:
plotly.offline.iplot({
    "data": [go.Scatter(x=fullData.PERatio, y=fullData.PayoutRatio, mode='markers')],
    "layout": go.Layout(yaxis_title="PayoutRatio", title="PayoutRatio vs. PERatio", height=300)
})
px.scatter(fullData,x='PERatio',y='MarketCap',title='MarketCap vs. PERatio',height=300)

In [12]:
# it seems we have outliers here, let's get rid of them
# applying the Median Absolute Deviation technique
# MAD = median|x - median.x|

PERatioMAD = np.median(np.abs(fullData['PERatio'] - np.median(fullData['PERatio'])))

# We can calculate the Modified Z-score like this:
MADZScores = (0.6745 * np.abs(fullData['PERatio'] - np.median(fullData['PERatio'])))/PERatioMAD

# Now we can calculate the score for each point of our sample! 
# As a rule of thumb, we'll use the score of 3.5 as our cut-off value; 
# This means that every point with a score above 3.5 will be considered an outlier.
fullDataNoOutliers = fullData[MADZScores < 3.5]

In [13]:
# have a look on relationships of numeric variables
px.scatter_matrix(fullDataNoOutliers,dimensions=numericalVar,color='GICSSector',height = 700)

In [14]:
# I think now it's good enough. But there are two big companies in IT sector, what are they?
fullDataNoOutliers[fullDataNoOutliers.MarketCap>1000000]

,Symbol,Security,SECfilings,GICSSector,GICSSubIndustry,Price,DividendYield,MarketCap,PERatio,PayoutRatio,Beta
49,AAPL,Apple Inc.,reports,Information Technology,"Technology Hardware, Storage & Peripherals",243.18,1.2,1098974.0,20.6,25.0,1.53
322,MSFT,Microsoft Corp.,reports,Information Technology,Systems Software,137.24,1.3,1047883.0,26.9,36.0,1.34


In [15]:
# the names and numbers look legit to me, I think they are not outliers
# OK, let's then do more data mining and data visualization work

# correlation heatmap between variables
corr = pd.DataFrame.corr(fullDataNoOutliers[numericalVar])

fig = go.Figure(data=go.Heatmap(z=corr,colorscale=plotly.colors.colorbrewer.RdBu_r[4:9],x=numericalVar,y=numericalVar))
fig.layout.update(xaxis_autorange='reversed', width=400, height=400, title = 'Correlation Matrix',title_x=0.5)
fig.show()

In [16]:
# dividend, payoutRatio, marketcap by sectors
px.scatter(fullDataNoOutliers, x='PayoutRatio', y='DividendYield', color='GICSSector', size='MarketCap',size_max=40,
          hover_name='Security', facet_col='GICSSector', facet_col_wrap=3, log_x=True, height = 800)

In [18]:
# we can see the difference is huge among sectors, pretty interesting
fig = px.scatter(fullDataNoOutliers, x='PayoutRatio', y='DividendYield', color='GICSSector', marginal_y="violin",
           marginal_x="box", trendline="ols")
fig.layout.update(showlegend=False)
fig.show()

In [23]:
# let's look at information in beta
#fullDataNoOutliers = fullDataNoOutliers.drop('BetaTo1',axis=1)

fullDataNoOutliers['BetaTo1'] = ''
fullDataNoOutliers.loc[fullDataNoOutliers[fullDataNoOutliers['Beta']<=1].index,'BetaTo1'] = '<=1'
fullDataNoOutliers.loc[fullDataNoOutliers[fullDataNoOutliers['Beta']>1].index,'BetaTo1'] = '>1'

fig = px.histogram(fullDataNoOutliers,x='GICSSector',barmode='group',y='BetaTo1',color='BetaTo1',
                   color_discrete_sequence=[px.colors.carto.Teal[1],px.colors.carto.Pinkyl[1]],height=400)
fig.layout.update(xaxis_autorange='reversed')
fig.show()

In [26]:
# we can see the beta distribution in different sectors
# is it affected by marketcap or sector?

# uniform marketCap
MktCap = fullDataNoOutliers['MarketCap']
MktCapUni = (MktCap - np.mean(MktCap))/np.std(MktCap,ddof=1)
fullDataNoOutliers['MarketCapUniformed'] = MktCapUni

fig = px.scatter(fullDataNoOutliers, x='MarketCapUniformed', y= 'Beta', color='GICSSector', trendline="ols", height=400)
fig.layout.update(showlegend=False)
fig.show()

In [25]:
# from the figure we can see there are downard and upward trands, and the r_square is really low
# but the beta distributed so different in different sectors

# click on legends to see different sectors

fig = go.Figure()
for i in set(fullDataNoOutliers['GICSSector']):
    fig.add_trace(go.Histogram(x=fullDataNoOutliers.loc[fullDataNoOutliers['GICSSector']==i,'Beta'], name=i))


# Overlay histograms
fig.update_layout(barmode='overlay',height=500)
# Reduce opacity to see each histograms
fig.update_traces(opacity=0.65)
fig.show()


In [41]:
# finally, let's find some pattern beyond sector
# clustering

estimator = KMeans(n_clusters=6)
estimator.fit(fullDataNoOutliers[['MarketCapUniformed','Beta']])

# visualise data
fullDataNoOutliers['cluster'] = estimator.labels_
centers = pd.DataFrame(estimator.cluster_centers_)
fig = px.scatter(fullDataNoOutliers,x='MarketCapUniformed', y= 'Beta', color='cluster', height=400)
fig.layout.update(coloraxis_showscale=False)
fig.show()